# Ecuaciones orbitas

## Objetivo

Crear un objeto que cree distintos tipos de orbitas en función de sus parámetros de entrada.

1. Creación de una órbita circular:
   - Le pasamos el attractor (planeta, masa, radio)
   - Le pasamos el radio de la órbita
   - Le pasamos inclinacion
   - Le pasamos anomalía verdadera inicial
   - Le pasamos RAAN
   - Si es Kepleriana

Nos debe devoler:
- Energía específica
- Momento angular específico
- mu
- función de trayectoria (r(theta))

In [242]:
import numpy as np

class ValidateOrbitParams:
    REQUIRED_PARAMS = ['attractor', 'distance', 'i', 'nu']
    OPTIONAL_PARAMS = {'e': 0, 'keplerian': True, 'a': None, 'r_p': None, 'r_a': None}

    def __init__(self, params):
        if not isinstance(params, dict):
            raise TypeError("Se espera un diccionario con los parámetros.")
        self.params = params
        self.validated = {}
        self.validate()
        self.validateRanges()

    def validateRanges(self):
        if not isinstance(self.validated['distance'], (int, float, np.floating, np.integer)):
            raise TypeError("El parámetro 'radius' debe ser un número.")
        if not isinstance(self.validated['i'], (int, float, np.floating, np.integer)):
            raise TypeError("El parámetro 'i' debe ser un número en radianes.")
        if not isinstance(self.validated['nu'], (int, float, np.floating, np.integer)):
            raise TypeError("El parámetro 'nu' debe ser un número en radianes.")
        if not isinstance(self.validated['e'], (int, float, np.floating, np.integer)):
            raise TypeError("El parámetro 'e' debe ser un número.")
        if not isinstance(self.validated['keplerian'], bool):
            raise TypeError("El parámetro 'keplerian' debe ser un booleano.")

        # Validaciones de rango
        if not (0 <= self.validated['i'] <= 2*np.pi):
            raise ValueError("El parámetro 'i' debe estar en radianes entre 0 y 2π.")
        if not (0 <= self.validated['nu'] <= 2*np.pi):
            raise ValueError("El parámetro 'nu' debe estar en radianes entre 0 y 2π.")
        if not (0 <= self.validated['e'] < 1):
            raise ValueError("La excentricidad 'e' debe estar entre 0 y 1.")
    
    def validate(self):
        # Validar parámetros obligatorios
        for key in self.REQUIRED_PARAMS:
            if key not in self.params:
                raise ValueError(f"El parámetro {key} es obligatorio")
            self.validated[key] = self.params[key]

        for key, default in self.OPTIONAL_PARAMS.items():
            self.validated[key] = self.params.get(key, default)
     

        return self.validated

In [266]:
ellipse_parameters = ["r_p", "r_a", "a", "b", "e", "p"]
EllipseParameters = namedtuple("Ellipse", ellipse_parameters)
class Ellipse:
    """
    Se puede inicializar con:
    - r_p y r_a (periastro y apoastro)
    - o a y e (semieje mayor y excentricidad)
    """
    def __init__(self, r_p=None, r_a=None, a=None, e=None):
        # Inicialización por periastro y apoastro
        if r_p is not None and r_a is not None:
            self.r_p = r_p
            self.r_a = r_a
            self.a = (r_a + r_p) / 2
            self.b = math.sqrt(r_a * r_p)
            self.e = (r_a - r_p) / (r_a + r_p)
            self.p = self.a * (1 - self.e**2)

        elif a is not None and e is not None:
            self.a = a
            self.e = e
            self.p = a * (1 - e**2)
            self.r_p = a * (1 - e)
            self.r_a = a * (1 + e)
            self.b = np.sqrt(a**2 * (1 - e**2))
        else:
            raise ValueError("Debes ingresar r_p y r_a o a y e")
    
    def r(self, theta):
        """Distancia radial en función del ángulo polar theta (radianes)"""
        return self.p / (1 + self.e * np.cos(theta))

    def info(self):
        return EllipseParameters(
            a = self.a * u.km,
            b= self.b * u.km,
            e = self.e,
            r_p = self.r_p * u.km,
            r_a = self.r_a * u.km,
            p = self.p * u.km,
        )
  

In [267]:
ellipse = Ellipse(a = 7000, e=0.2).info()
ellipse.a
# print("r(90°) =", ellipse.r(math.radians(90)))


<Quantity 7000. km>

In [330]:
import numpy as np
import astropy.units as u
from astropy.constants import G
from dataclasses import dataclass, asdict
from pprint import pprint

# Dataclass para los parámetros de la órbita
@dataclass
class OrbitInfo:
    attractor: str
    distance: float
    inclination_rad: float
    true_anomaly_rad: float
    eccentricity: float
    keplerian: bool
    specific_e: float
    h_mag: float
    v_p: float
    v_a: float


class EllipticalOrbit:
    def __init__(self, params):
        validated_params = ValidateOrbitParams(params).validated

        self.attractor = validated_params['attractor']
        self.r = validated_params['distance'] * u.km
        self.i = validated_params['i']
        self.nu = validated_params['nu']
        self.ellipse_params = Ellipse(
            r_p=validated_params.get('r_p'),
            r_a=validated_params.get('r_a'),
            a=validated_params.get('a'),
            e=validated_params.get('e')
        ).info()
        self.keplerian = validated_params['keplerian']
        self.mu = (G * self.attractor.mass).to(u.km**3 / u.s**2)

    def specific_energy(self):
        return (-self.mu / (2 * self.ellipse_params.a)).to(u.km**2 / u.s**2)

    def h_mag(self):
        return np.sqrt(self.mu * self.ellipse_params.a * (1 - self.ellipse_params.e**2)).to(u.km**2 / u.s)

    def v_p(self):
        v = np.sqrt(self.mu * (2/self.ellipse_params.r_p - 1/self.ellipse_params.a))
        return v.to(u.km/u.s)

    def v_a(self):
        v = np.sqrt(self.mu * (2/self.ellipse_params.r_a - 1/self.ellipse_params.a))
        return v.to(u.km/u.s)

    def info(self):
        """Devuelve un OrbitInfo (dataclass)"""
        return OrbitInfo(
            attractor=self.attractor.name if hasattr(self.attractor, "name") else str(self.attractor),
            distance=self.r.value,
            inclination_rad=self.i,
            true_anomaly_rad=self.nu,
            eccentricity=self.ellipse_params.e,
            keplerian=self.keplerian,
            specific_e=self.specific_energy().value,
            h_mag=self.h_mag().value,
            v_p=self.v_p().value,
            v_a=self.v_a().value
        )



In [334]:
params = {
    "attractor": Earth,
    "distance": 700,
    "i": np.radians(30),
    "nu": np.radians(45),
    "a": 7000,
    "e": 0.1,
}

orbit = EllipticalOrbit(params)
orbit_info = orbit.info()

pprint(asdict(orbit_info))


6.825661663374264
{'attractor': 'Earth',
 'distance': 700.0,
 'eccentricity': 0.1,
 'h_mag': 52557.59480798184,
 'inclination_rad': 0.5235987755982988,
 'keplerian': True,
 'specific_e': -28.471457142857144,
 'true_anomaly_rad': 0.7853981633974483,
 'v_a': 6.825661663374264,
 'v_p': 8.342475366346324}


In [46]:
from astropy.constants import G, M_jup, R_jup, M_earth
from poliastro.bodies import Earth
